**This is example of how to trace model with jit and export it to the onnx**

In [1]:
from rl_games.torch_runner import Runner
import ray
import yaml
import torch
import matplotlib.pyplot as plt
import gym
from IPython import display
import numpy as np
import onnx
import onnxruntime as ort
%matplotlib inline

/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):


In [2]:
ray.init(object_store_memory=1024*1024*1000)

/home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


RayContext(dashboard_url='', python_version='3.9.12', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '10.2.168.57', 'raylet_ip_address': '10.2.168.57', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-09-16_13-18-13_562612_1202937/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-09-16_13-18-13_562612_1202937/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-09-16_13-18-13_562612_1202937', 'metrics_export_port': 49137, 'gcs_address': '10.2.168.57:46385', 'address': '10.2.168.57:46385', 'node_id': '1616c3c24137e3b7904ed818df91e989e9d6bfc948d684e8c4775004'})

In [3]:
config_name = '../rl_games/configs/ppo_pendulum_torch.yaml'

In [ ]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    config['params']['config']['normalize_input'] = True
    config['params']['config']['max_epochs'] = 500
    config['params']['config']['full_experiment_name'] = 'pendulum_onnx'
    config['params']['config']['save_best_after'] = 10
runner = Runner()
runner.load(config)
runner.run({
    'train': True,
})

self.seed = 1663359495
Started to train
Exact experiment name requested from command line: pendulum_onnx


(RayWorker pid=1203304) /home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
(RayWorker pid=1203304)   deprecation(
(RayWorker pid=1203308) /home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
(RayWorker pid=1203308)   deprecation(
(RayWorker pid=1203305) /home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
(RayWorker pid=1203305)   deprecation(
(RayWorker pid=1203307) /home/viktorm/anaconda3/envs/warp39/lib/python3.9/site-packages/gym/core.py:172: Depr

current training device: cuda:0
build mlp: 3
build mlp: 3
RunningMeanStd:  (3,)
fps step: 3240 fps step and policy inference: 1395 fps total: 1367 epoch: 1/500
fps step: 3240 fps step and policy inference: 2521 fps total: 2438 epoch: 2/500
fps step: 3243 fps step and policy inference: 2506 fps total: 2423 epoch: 3/500
fps step: 3267 fps step and policy inference: 2518 fps total: 2435 epoch: 4/500
fps step: 3277 fps step and policy inference: 2537 fps total: 2427 epoch: 5/500
fps step: 3107 fps step and policy inference: 2397 fps total: 2314 epoch: 6/500
fps step: 3281 fps step and policy inference: 2528 fps total: 2435 epoch: 7/500
fps step: 3154 fps step and policy inference: 2440 fps total: 2346 epoch: 8/500
fps step: 3084 fps step and policy inference: 2402 fps total: 2324 epoch: 9/500
fps step: 3363 fps step and policy inference: 2575 fps total: 2480 epoch: 10/500
saving next best rewards:  [-1216.8058]
=> saving checkpoint 'runs/pendulum_onnx/nn/pendulum.pth'
fps step: 3270 fps st

fps step: 3233 fps step and policy inference: 2490 fps total: 2403 epoch: 67/500
fps step: 3340 fps step and policy inference: 2572 fps total: 2485 epoch: 68/500
saving next best rewards:  [-361.25574]
=> saving checkpoint 'runs/pendulum_onnx/nn/pendulum.pth'
fps step: 3344 fps step and policy inference: 2572 fps total: 2479 epoch: 69/500
saving next best rewards:  [-330.30405]
=> saving checkpoint 'runs/pendulum_onnx/nn/pendulum.pth'
fps step: 3224 fps step and policy inference: 2491 fps total: 2407 epoch: 70/500
fps step: 3324 fps step and policy inference: 2557 fps total: 2468 epoch: 71/500
saving next best rewards:  [-302.3561]
=> saving checkpoint 'runs/pendulum_onnx/nn/pendulum.pth'
fps step: 3310 fps step and policy inference: 2553 fps total: 2460 epoch: 72/500
saving next best rewards:  [-281.5465]
=> saving checkpoint 'runs/pendulum_onnx/nn/pendulum.pth'
fps step: 3230 fps step and policy inference: 2477 fps total: 2398 epoch: 73/500
fps step: 3279 fps step and policy inferenc

In [ ]:
class ModelWrapper(torch.nn.Module):
    '''
    Main idea is to ignore outputs which we don't need from model
    '''
    def __init__(self, model):
        torch.nn.Module.__init__(self)
        self._model = model
        
        
    def forward(self,input_dict):
        input_dict['obs'] = self._model.norm_obs(input_dict['obs'])
        '''
        just model export doesn't work. Looks like onnx issue with torch distributions
        thats why we are exporting only neural network
        '''
        #print(input_dict)
        #output_dict = self._model.a2c_network(input_dict)
        #input_dict['is_train'] = False
        #return output_dict['logits'], output_dict['values']
        return self._model.a2c_network(input_dict)

In [ ]:
agent = runner.create_player()
agent.restore('runs/pendulum_onnx/nn/pendulum.pth')

import rl_games.algos_torch.flatten as flatten
inputs = {
    'obs' : torch.zeros((1,) + agent.obs_shape).to(agent.device),
    'rnn_states' : agent.states,
}

with torch.no_grad():
    adapter = flatten.TracingAdapter(ModelWrapper(agent.model), inputs,allow_non_tensor=True)
    traced = torch.jit.trace(adapter, adapter.flattened_inputs,check_trace=False)
    flattened_outputs = traced(*adapter.flattened_inputs)
    print(flattened_outputs)
    
torch.onnx.export(traced, *adapter.flattened_inputs, "pendulum.onnx", verbose=True, input_names=['obs'], output_names=['mu','log_std', 'value'])

onnx_model = onnx.load("pendulum.onnx")

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

In [ ]:
ort_model = ort.InferenceSession("pendulum.onnx")

outputs = ort_model.run(
    None,
    {"obs": np.zeros((1, 3)).astype(np.float32)},
)
print(outputs)

In [ ]:
is_done = False
env = agent.env
obs = env.reset()
#prev_screen = env.render(mode='rgb_array')
#plt.imshow(prev_screen)
total_reward = 0
num_steps = 0
while not is_done:
    outputs = ort_model.run(None, {"obs": np.expand_dims(obs, axis=0).astype(np.float32)},)
    action = outputs[0].squeeze(1)
    obs, reward, done, info = env.step(action)
    total_reward += reward
    num_steps += 1
    is_done = done
    screen = env.render(mode='rgb_array')
    #plt.imshow(screen)
    #display.display(plt.gcf())    
    #display.clear_output(wait=True)
print(total_reward, num_steps)
#ipythondisplay.clear_output(wait=True)